Write a Spark GraphX program to perform few analysis steps for sample US airports. Conduct the following steps:
Steps & Hints: 
1.	Generate a new RDD containing few selected US airport information, such as: (“BWI”, “Baltimore Washington Airport”, “JFK”, “NYC Airport”, “MIA”, “Miami Airport”, “LAX”, “Los Angeles Airport”, etc.)
2.	Limit your airport selection to 10 airports
3.	Generate random data for flights between two random airports (selected from the RDD above). Random data generated can be in the form (src, dst, flight number), (“BWI”, “LAX”, “1552”), etc.
4.	You may use similar algorithm from the previous homework, make sure you generate 200+ flights (if you have a randomize function that generates one flight data information, then call the function 200 times)
5.	Generate the relationship graph
6.	Find the airport with the largest number of departure flight
7.	Find the airport with the largest number of arriving flight
8.	Find the busiest airport
9.	Note: Illustrate your work and display your results (to get full grade)


In [1]:
import pyspark
#import spark
from pyspark import SparkContext
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
sc = SparkContext()

In [3]:
 rdd=sc.textFile("AirportCode.csv")

In [4]:
rdd.collect()

['BWI,Baltimore Airport',
 'IAD,International Airport of Dulles',
 'NYA,NewYork Airport',
 'FIA,Florida International Airport',
 'INA,Indiana Airport',
 'VIA,Virginia International Airport',
 'KSA,Kansas State Airport',
 'TIA,Texas International Airport',
 'PNA,Pennsylvania Airport',
 'NJA,NewJersey Airport']

In [5]:
mappedRdd= rdd.map(lambda x:x.split(";"))

In [6]:
mappedRdd.collect()

[['BWI,Baltimore Airport'],
 ['IAD,International Airport of Dulles'],
 ['NYA,NewYork Airport'],
 ['FIA,Florida International Airport'],
 ['INA,Indiana Airport'],
 ['VIA,Virginia International Airport'],
 ['KSA,Kansas State Airport'],
 ['TIA,Texas International Airport'],
 ['PNA,Pennsylvania Airport'],
 ['NJA,NewJersey Airport']]

In [7]:
list1 = mappedRdd.collect()

In [8]:
type(list1)

list

In [9]:
mappedRdd.take(11)

[['BWI,Baltimore Airport'],
 ['IAD,International Airport of Dulles'],
 ['NYA,NewYork Airport'],
 ['FIA,Florida International Airport'],
 ['INA,Indiana Airport'],
 ['VIA,Virginia International Airport'],
 ['KSA,Kansas State Airport'],
 ['TIA,Texas International Airport'],
 ['PNA,Pennsylvania Airport'],
 ['NJA,NewJersey Airport']]

In [10]:
from random import randint
import random

In [11]:
df1=[]

In [12]:
#List = ['BWI','IAD','NYA','FIA','INA','VIA','KSA','TIA','PNA','NJA']
lis =['Journey']
for n in range(200):
    df1.append({'FlightNo': ((randint(1000, 9999))),
              'Source' : random.choice(list1),
               'Destination' : random.choice(list1),
                'Relationship':random.choice(lis)
               })

In [13]:
df1=pd.DataFrame(df1)

In [14]:
df1.head(5)

,Destination,FlightNo,Relationship,Source
0,"[TIA,Texas International Airport]",1517,Journey,"[FIA,Florida International Airport]"
1,"[BWI,Baltimore Airport]",7662,Journey,"[INA,Indiana Airport]"
2,"[VIA,Virginia International Airport]",9642,Journey,"[INA,Indiana Airport]"
3,"[IAD,International Airport of Dulles]",5177,Journey,"[PNA,Pennsylvania Airport]"
4,"[KSA,Kansas State Airport]",9704,Journey,"[INA,Indiana Airport]"


In [15]:
df1.to_csv("data.csv")

In [16]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer,StopWordsRemover,Word2Vec,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.sql.functions import udf
from pyspark.sql.types import *

import re
import shutil

In [17]:
pip show pyspark

Name: pyspark
Version: 2.4.5
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: c:\users\pushy\anaconda3\lib\site-packages
Requires: py4j
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [18]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,Word2Vec
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [19]:
import re
import numpy as np
import pandas as pd
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [20]:
import folium
from os import path, getcwd
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [21]:
spark = SparkSession.builder.appName('SentimentClassifierCreationWithSparkML').getOrCreate()

'''
Unpacking Archive data file 
'''
#shutil.unpack_archive('training.1600000.processed.noemoticon.csv.tar.gz', 'datasets')
#print('Archive file unpacked successfully.')

df3 = spark.read.csv('data.csv',inferSchema=True)

In [22]:
print('Total Number of records in df after deleting duplicate and null records : ',df3.count())
df3 = df3.dropna()

Total Number of records in df after deleting duplicate and null records :  201


In [23]:
df3.show(5)

+---+--------------------+----+-------+--------------------+
|_c0|                 _c1| _c2|    _c3|                 _c4|
+---+--------------------+----+-------+--------------------+
|  0|['TIA,Texas Inter...|1517|Journey|['FIA,Florida Int...|
|  1|['BWI,Baltimore A...|7662|Journey|['INA,Indiana Air...|
|  2|['VIA,Virginia In...|9642|Journey|['INA,Indiana Air...|
|  3|['IAD,Internation...|5177|Journey|['PNA,Pennsylvani...|
|  4|['KSA,Kansas Stat...|9704|Journey|['INA,Indiana Air...|
+---+--------------------+----+-------+--------------------+
only showing top 5 rows



In [24]:
#Destination	FlightNo	Source
df3 = df3.withColumnRenamed('_c0','Index').withColumnRenamed('_c1','Destination').withColumnRenamed('_c2','FlightNo').withColumnRenamed('_c3','Relationships').withColumnRenamed('_c4','Source')
df3.show(5)

+-----+--------------------+--------+-------------+--------------------+
|Index|         Destination|FlightNo|Relationships|              Source|
+-----+--------------------+--------+-------------+--------------------+
|    0|['TIA,Texas Inter...|    1517|      Journey|['FIA,Florida Int...|
|    1|['BWI,Baltimore A...|    7662|      Journey|['INA,Indiana Air...|
|    2|['VIA,Virginia In...|    9642|      Journey|['INA,Indiana Air...|
|    3|['IAD,Internation...|    5177|      Journey|['PNA,Pennsylvani...|
|    4|['KSA,Kansas Stat...|    9704|      Journey|['INA,Indiana Air...|
+-----+--------------------+--------+-------------+--------------------+
only showing top 5 rows



In [25]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
#sqlContext = SQLContext(sc)
#schema = StructType([StructField(str(i), StringType()) for i in range(10)])

#df = sqlContext.createDataFrame(mappedRdd)

In [26]:
hasattr(mappedRdd, "toDF")

True

In [27]:
spark = SparkSession(sc)
hasattr(mappedRdd, "toDF")
## True

True

In [28]:
data=mappedRdd.toDF()

In [29]:
data.show()

+--------------------+
|                  _1|
+--------------------+
|BWI,Baltimore Air...|
|IAD,International...|
| NYA,NewYork Airport|
|FIA,Florida Inter...|
| INA,Indiana Airport|
|VIA,Virginia Inte...|
|KSA,Kansas State ...|
|TIA,Texas Interna...|
|PNA,Pennsylvania ...|
|NJA,NewJersey Air...|
+--------------------+



In [30]:
data = data.withColumnRenamed('_1','Airports')
data.show()

+--------------------+
|            Airports|
+--------------------+
|BWI,Baltimore Air...|
|IAD,International...|
| NYA,NewYork Airport|
|FIA,Florida Inter...|
| INA,Indiana Airport|
|VIA,Virginia Inte...|
|KSA,Kansas State ...|
|TIA,Texas Interna...|
|PNA,Pennsylvania ...|
|NJA,NewJersey Air...|
+--------------------+



In [ ]:
pip install graphframes

In [ ]:
df3.show(5)

In [ ]:
import graphframes

In [ ]:
pip install graphframes

In [ ]:
from graphframes import *

In [ ]:
from pyspark.sql import *

In [ ]:
import numpy as np

In [ ]:
from graphframes import GraphFrame

In [ ]:
from functools import reduce
from pyspark.sql.functions import col, lit, when
from graphframes import *

In [ ]:
vertices = data

In [ ]:
edges = df3

In [ ]:
from graphframes import *

# Create Vertices (airports) and Edges (flights)

tripEdges = df3.select("Index","Destination","FlightNo","Relationships","Source")
tripVertices = data.select("Airports")

In [ ]:
#tripEdges = departureDelays.select(“tripid”, “delay”, “src”, “dst”, “city_dst”, “state_dst”)
# This GraphFrame builds upon the vertices and edges based on our trips (flights)

tripGraph = GraphFrame(tripVertices, tripEdges)

In [ ]:
#print “Airports: %d” % tripGraph.vertices.count()

#print “Trips: %d” % tripGraph.edges.count()

In [ ]:
#wget https://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.7.0-spark2.4-s_2.11/graphframes-0.7.0-spark2.4-s_2.11.jar -P /path-to-spark/spark/jars/

In [ ]:
spark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("HELK Graphs") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
!pip install --user pixiedust

In [ ]:
 !pip install --user --upgrade --no-deps pixiedust

In [ ]:
from pixiedust.packageManager import PackageManager
pkg=PackageManager()
pkg.installPackage("graphframes:graphframes:0")
pkg.printAllPackages()
 
sqlContext=SQLContext(sc)

In [ ]:
#import the display module
from pixiedust.display import *
#import the Graphs example
#from graphframes.examples import Graphs
#create the friends example graph
#g=Graphs(sqlContext).friends()
#use the pixiedust display
#display(g)

In [ ]:
df3.orderBy("Source").show()

In [ ]:
df3.orderBy("Destination").show()

In [ ]:
df3.groupBy('Destination').max().show()

In [ ]:
group_data = df3.groupBy("Source")
group_data.agg({'Destination':'count'}).show()

In [ ]:
group_data1 = df3.groupBy("Destination")
group_data1.agg({'Source':'count'}).show()

In [ ]:
#df3.agg(corr("Destination", "Source").alias('correlation')).collect()

In [ ]:
df_Source=group_data1.agg({'Source':'count'}).toPandas()
df_Source

In [ ]:
d2=df_Source.rename(columns={'Destination':'All','count(Source)':'Count'})
d2

In [ ]:
x=['INA','IAD','VIA','PNA','KSA','BWI','NJA','TIA','FIA','NYA']
rect=df_Source.loc[:,['Destination','count(Source)']].plot.bar(figsize=(10,7), fontsize=13)
plt.xticks(range(10),['INA','IAD','VIA','PNA','KSA','BWI','NJA','TIA','FIA','NYA'])
plt.xlabel('Destinations',fontsize=18) 
plt.ylabel('sum of dstination to sources', fontsize=18) 
plt.title('Airport with the largest number of arrival flight', fontsize=23)

In [ ]:
df_Dest=group_data.agg({'Destination':'count'}).toPandas()
d1=df_Dest.rename(columns={'Source':'All','count(Destination)':'Count'})
d1

In [ ]:
x=['INA','IAD','VIA','PNA','KSA','BWI','NJA','TIA','FIA','NYA']
rect=df_Dest.loc[:,['Source','count(Destination)']].plot.bar(figsize=(10,7), fontsize=13)
plt.xticks(range(10),['INA','IAD','VIA','PNA','KSA','BWI','NJA','TIA','FIA','NYA'])
plt.xlabel('Sources',fontsize=18) 
plt.ylabel('sum of Source to destination', fontsize=18) 
plt.title('Airport with the largest number of Departure', fontsize=23)

In [ ]:
#df_Dest.rename(columns={'Source':'Source','count(Destination)':'Count'})

In [ ]:
dff = pd.merge(d1, d2, on='All',how='inner')

In [ ]:
dff

In [ ]:
dff['BusyCount'] = dff['Count_x'] + dff['Count_y']

In [ ]:
dff

In [ ]:
x=['INA','IAD','VIA','PNA','KSA','BWI','NJA','TIA','FIA','NYA']
rect=dff.loc[:,['BusyCount']].plot.bar(figsize=(10,7), fontsize=13)
plt.xticks(range(10),['INA','IAD','VIA','PNA','KSA','BWI','NJA','TIA','FIA','NYA'])
plt.xlabel('Airports',fontsize=18) 
plt.ylabel('Count of arrivals and departures', fontsize=18) 
plt.title('The busiest airport', fontsize=23)

In [ ]:
n_groups = 10
fig, ax = plt.subplots(figsize=(13,5))
bar_width = 0.20
opacity = 0.8
index = np.arange(n_groups)

rects1 = plt.bar(index - bar_width, dff['Count_x'], bar_width,alpha=opacity,color='b',label='Sources')
rects2 = plt.bar(index, dff['Count_y'], bar_width,alpha=opacity,color='g',label='Destinations')
rects3 = plt.bar(index + bar_width, dff['BusyCount'], bar_width,alpha=opacity,color='r',label='Busy Airport')
plt.xlabel('Airports', fontsize=18)
plt.ylabel('Total Counts', fontsize=18)
plt.title('COunts of Departure Arrival and busy airport',fontsize=23)
plt.xticks(index + bar_width, ('INA','IAD','VIA','PNA','KSA','BWI','NJA','TIA','FIA','NYA'))
plt.legend()
plt.tight_layout()
plt.show()
